In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Przygotowanie danych do uczenia i podzielenie na zbiór uczący i zbiór testowy

In [2]:
data = pd.read_pickle("../../../data/03_training_set/training_set.pkl")
X = data.drop(['new_watchers'], axis=1)
y = data['new_watchers']

Standaryzacja danych - pozwala na poprawienie wyników uczenia i przyspieszenie obliczeń

In [3]:
s = StandardScaler()
X = s.fit_transform(X)
X

array([[-1.10809584, -0.03015276, -0.02863816, ...,  7.10688161,
        -0.01097345, -0.05105899],
       [-1.01069671, -0.03015276, -0.02863816, ...,  7.10688161,
        -0.01097345, -0.05105899],
       [-0.91329758, -0.03015276, -0.02863816, ...,  7.10688161,
        -0.01097345, -0.05105899],
       ...,
       [-1.01069671, -0.03015276, -0.02863816, ..., -0.14070869,
        -0.01097345, -0.05105899],
       [-0.91329758, -0.03015276, -0.02863816, ..., -0.14070869,
        -0.01097345, -0.05105899],
       [-0.81589845, -0.03015276, -0.02863816, ..., -0.14070869,
        -0.01097345, -0.05105899]])

Podział na dane treningowe i dane testowe, standardowo w stosunku 80:20

In [4]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.30, random_state=42)

## Las losowy

In [40]:
from sklearn.ensemble import RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=100, verbose=1, random_state=7)

In [41]:
random_forest.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  9.2min finished


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=7, verbose=1, warm_start=False)

In [38]:
training_score = random_forest.score(X_train, y_train)
test_score = random_forest.score(X_test, y_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


In [39]:
training_score

0.6620209838297832

In [34]:
test_score

-0.11649357048677356

## Sieć neuronowa

In [44]:
from sklearn.metrics import r2_score, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.
D:\Programy\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Programy\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Programy\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Programy\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: Future

In [46]:
def baseline_model():
    model = Sequential()
    model.add(Dense(100,input_dim=40, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))    
    model.add(Dense(80, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(60, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))   
    model.add(Dense(40, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(20, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='relu', kernel_initializer='glorot_normal'))
    model.add(Dropout(0.2))
    model.add(Dense(1, kernel_initializer='glorot_normal'))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

estimator = KerasRegressor(build_fn=baseline_model, nb_epoch=200, epochs=10, batch_size=32, verbose=True)

estimator.fit(X_train, y_train)


Epoch 1/10
1267420/1267420 [==============================] - 82s 65us/step - loss: 461.9692
Epoch 2/10
1267420/1267420 [==============================] - 81s 64us/step - loss: 461.8980
Epoch 3/10
1267420/1267420 [==============================] - 81s 64us/step - loss: 464.6963
Epoch 4/10
1267420/1267420 [==============================] - 85s 67us/step - loss: 460.9349
Epoch 5/10
1267420/1267420 [==============================] - 76s 60us/step - loss: 459.1341
Epoch 6/10
1267420/1267420 [==============================] - 80s 63us/step - loss: 458.5971
Epoch 7/10
1267420/1267420 [==============================] - 75s 59us/step - loss: 459.4560
Epoch 8/10
1267420/1267420 [==============================] - 75s 59us/step - loss: 459.0452
Epoch 9/10
1267420/1267420 [==============================] - 77s 61us/step - loss: 458.2937
Epoch 10/10
1267420/1267420 [==============================] - 81s 64us/step - loss: 459.7704


In [47]:
train_pred = estimator.predict(X_train)
test_pred = estimator.predict(X_test)

training_score = r2_score(y_train, train_pred)
test_score = r2_score(y_test, test_pred)

543181/543181 [==============================] - 8s 14us/step


In [48]:
training_score

0.027378655036948918

In [49]:
test_score

0.07758219040096181